In [ ]:
# 2024-09-08
# Wonseok Hwang
# CC-BY-NC 4.0 International License

In [1]:

!pip install --quiet datasets
!pip install --quiet transformers
!pip install --quiet lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.

# 1. Dataset

# 2. Training

In [2]:
import datasets
import torch
from transformers import AutoModel, AutoTokenizer

dataset_card = "lbox/lbox_open"
task = "casename_classification"
data = datasets.load_dataset(dataset_card, task)
data_t = data["train"]
CASENAMES = tuple(sorted(set(data_t["casename"])))

backbone_card = "distilbert-base-multilingual-cased"
backbone = AutoModel.from_pretrained(backbone_card)
tokenizer = AutoTokenizer.from_pretrained(backbone_card)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test2 split:   0%|          | 0/1294 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
def my_collate_fn(batch):
    xs = [x["facts"] for x in batch]
    ys = [x["casename"] for x in batch]
    ys = [CASENAMES.index(y) for y in ys]


    input_features = tokenizer(xs, padding=True, truncation=True, return_tensors="pt")
    # input_ids = input_features["input_ids"].to(device)
    input_ids = input_features["input_ids"]
    # attention_mask = input_features["attention_mask"].to(device)
    attention_mask = input_features["attention_mask"]
    # label = torch.tensor(ys).to(device)
    label = torch.tensor(ys)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "label": label,
    }


dataloader_t = torch.utils.data.DataLoader(
    data["train"],
    batch_size=36,
    shuffle=True,
    collate_fn=my_collate_fn,
)

dataloader_v = torch.utils.data.DataLoader(
    data["validation"],
    batch_size=36,
    shuffle=False,
    collate_fn=my_collate_fn,
)

In [4]:
class MyModel(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.pooler = torch.nn.Linear(768, 100)
    def forward(self, input_ids, attention_mask):
        encoded = self.backbone(input_ids, attention_mask)
        last = encoded.last_hidden_state # [batch, 768]
        logit = self.pooler(last[:, 0]) # [batch, 100]
        return logit


model = MyModel(backbone)
# backbone = backbone.to(device)
# model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

In [5]:
# ref) https://lightning.ai/docs/fabric/stable/

# !pip install -U torch_xla # to use tpu # should be installed after loading the model
# !pip install -U cloud-tpu-client -> not working

import os
import torch
import lightning as L

# torch.manual_seed(0)
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
L.Fabric.seed_everything(0)

# Fabric은 PyTorch Lightning의 기능 중 하나로, 다양한 하드웨어에서 쉽게 모델을 훈련하고 관리할 수 있도록 도와주는 래퍼
fabric = L.Fabric(accelerator="auto") # cpu, gpu, tpu
# fabric = L.Fabric(accelerator="tpu") # tpu is not supported yet.
# fabric = L.Fabric(accelerator="cpu")
# fabric = L.Fabric(accelerator="gpu", devices=4, strategy="dp")
fabric.launch()

model, optimizer = fabric.setup(model, optimizer)
dataloader_t = fabric.setup_dataloaders(dataloader_t) # fabric형태로 변환
dataloader_v = fabric.setup_dataloaders(dataloader_v)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")


INFO: Seed set to 0
INFO:lightning.fabric.utilities.seed:Seed set to 0


In [ ]:
from tqdm import tqdm

model = model.train()
for epoch in (range(2)):
    tot_loss = 0
    for i, batch in enumerate(tqdm(dataloader_t)):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        label = batch["label"]

        logit = model.forward(input_ids, attention_mask)

        optimizer.zero_grad()
        loss1 = loss_fn(logit, label)
        tot_loss += loss1.item()
        # loss1.backward()
        fabric.backward(loss1)
        optimizer.step()
        if i % 10 == 0:
            print(f"epoch: {epoch}, batch: {i}, loss: {tot_loss / (i+1)}")

    print(f"epoch: {epoch}, tot_loss: {tot_loss}")

  0%|          | 1/223 [00:04<14:54,  4.03s/it]

epoch: 0, batch: 0, loss: 4.626089096069336


  5%|▍         | 11/223 [00:17<05:06,  1.45s/it]

epoch: 0, batch: 10, loss: 4.649686986749822


  9%|▉         | 21/223 [00:32<04:53,  1.45s/it]

...

 27%|██▋       | 61/223 [01:31<04:02,  1.50s/it]

epoch: 0, batch: 60, loss: 4.600693499455686


 32%|███▏      | 71/223 [01:46<03:48,  1.51s/it]

epoch: 0, batch: 70, loss: 4.5839460601269355


 34%|███▎      | 75/223 [01:52<03:42,  1.51s/it]

In [ ]:
# device = "cpu"
# model = model.to(device)
cnt = 0
model = model.eval()
for batch in dataloader_v:
  cnt += 1
  # xs = batch["facts"]
  # ys = batch["casename"]
  # ys = [CASENAMES.index(y) for y in ys]

#   input_features = tokenizer(xs, padding=True, truncation=True, return_tensors="pt")
#   input_ids = input_features["input_ids"].to(device)
#   attention_mask = input_features["attention_mask"].to(device)
#   label = torch.tensor(ys).to(device)
  input_ids = batch["input_ids"]
  attention_mask = batch["attention_mask"]
  label = batch["label"]

  logit = model.forward(input_ids, attention_mask)
  preds = logit.argmax(-1)
  print(f"GT: {label}")
  print(f"PR: {preds}")
  if cnt > 10:
    break